In [1]:
# pip install pandas
# pip3 install tabulate

In [1]:
import pandas as pd
from tabulate import tabulate
import os
from datetime import datetime

print(os.getcwd())

c:\Users\Evan\Desktop\4442\project


In [26]:
os.chdir('..')
print(os.getcwd())

c:\Users\Evan\Desktop\4442\project\raw_data


In [2]:
os.chdir('raw_data')
os.chdir('channels')
path = os.getcwd()
print(path)

c:\Users\Evan\Desktop\4442\project\raw_data\channels


In [28]:
# change file names to date ranges only
files = [ f for f in os.listdir(path) if os.path.isfile(os.path.join(path,f)) ]
for f in files:
    os.rename(os.path.join(path,f),os.path.join(path, f.replace('Analytics Master Overview ', '').lower())) 
    #print (f)

In [5]:
files = [ f for f in os.listdir(path) if os.path.isfile(os.path.join(path,f)) ]
df = pd.DataFrame()
week = 1
for f in files:
    temp = pd.read_csv(f, skiprows=6, skipfooter=18, engine='python')
    temp = temp.drop(columns=['Users', 'New Users', 'Bounce Rate', 'Pages / Session', 'Link Click - Apply (Goal 13 Conversion Rate)', 'Link Click - Apply (Goal 13 Value)']) # remove unnecessary columns
    #print(temp.columns)
    temp.columns = ['Source', 'Sessions', 'Avg_Duration', 'Grad_Apply'] # shorten column names
    
    # add date range from file name
    temp['ID'] = week
    week = week + 1
    temp['Begin Date'] = datetime.strptime(f[0:8], '%Y%m%d').date()
    temp['End Date'] = datetime.strptime(f[9:17], '%Y%m%d').date()
    df = pd.concat([df, temp])
    

In [6]:
print(tabulate(df, showindex=False, headers=df.columns))

Source          Sessions    Avg_Duration      Grad_Apply    ID  Begin Date    End Date
--------------  ----------  --------------  ------------  ----  ------------  ----------
Organic Search  9,915       00:02:40                  17     1  2021-10-16    2021-10-31
Direct          6,353       00:02:07                  13     1  2021-10-16    2021-10-31
(Other)         5,157       00:00:37                   2     1  2021-10-16    2021-10-31
Paid Search     3,753       00:01:24                  13     1  2021-10-16    2021-10-31
Display         3,174       00:00:15                   0     1  2021-10-16    2021-10-31
Email           1,031       00:02:01                   0     1  2021-10-16    2021-10-31
Referral        569         00:02:11                   3     1  2021-10-16    2021-10-31
Social          263         00:01:33                   0     1  2021-10-16    2021-10-31
nan             30,215      00:01:45                  48     1  2021-10-16    2021-10-31
Organic Search  10,307 

In [63]:
# Combine '(Other)' into 'Direct'
df['Source'] = df['Source'].replace('(Other)', 'Direct')

# Split df into separate DataFrames based on 'Source'
source_groups = df.groupby('Source')

# Create separate DataFrames for each source
source_dfs = []
for source, group in source_groups:
    source_dfs.append(group.copy())

# Access individual DataFrames for each source
direct_df = source_dfs[0]          # DataFrame for 'Direct' (and '(Other)')
#direct_df = direct_df.drop(columns=['Source'])
display_df = source_dfs[1]         # DataFrame for 'Display'
email_df = source_dfs[2]           # DataFrame for 'Email'
organic_search_df = source_dfs[3]  # DataFrame for 'Organic Search'
paid_search_df = source_dfs[4]     # DataFrame for 'Paid Search'
referral_df = source_dfs[5]        # DataFrame for 'Referral'
social_df = source_dfs[6]          # DataFrame for 'Social'

In [64]:
# Merge 'Direct' and '(Other)' results by ID
direct_df['Sessions'] = direct_df['Sessions'].str.replace(',', '').astype(int)
direct_df['Avg_Duration'] = pd.to_timedelta(direct_df['Avg_Duration'])
direct_df = direct_df.groupby('ID').agg({
    'Source': 'first',
    'Sessions': 'sum',
    'Avg_Duration': 'sum',
    'Grad_Apply': 'sum',
    'Begin Date': 'max',
    'End Date': 'max'
}).reset_index()
direct_df['Avg_Duration'] = direct_df['Avg_Duration'].apply(lambda x: str(x).split()[-1])

column_order = ['Source', 'Sessions', 'Avg_Duration', 'Grad_Apply', 'ID', 'Begin Date', 'End Date']
direct_df = direct_df.reindex(columns=column_order)



In [67]:
print_df = direct_df
print(tabulate(print_df, showindex=False, headers=print_df.columns))

  ID  Source      Sessions  Avg_Duration      Grad_Apply  Begin Date    End Date
----  --------  ----------  --------------  ------------  ------------  ----------
   1  Direct         11510  00:02:44                  15  2021-10-16    2021-10-31
   2  Direct          9902  00:02:38                  63  2021-11-01    2021-11-15
   3  Direct          7699  00:02:21                  71  2021-11-16    2021-11-30
   4  Direct          8284  00:02:29                  76  2021-12-01    2021-12-15
   5  Direct          5363  00:03:08                  57  2021-12-16    2021-12-31
   6  Direct         11560  00:02:21                 101  2022-01-01    2022-01-15
   7  Direct         13065  00:02:25                  85  2022-01-16    2022-01-31
   8  Direct          7891  00:02:29                  68  2022-02-01    2022-02-14
   9  Direct          8728  00:02:33                  72  2022-02-15    2022-02-28
  10  Direct          8280  00:02:59                  87  2022-03-01    2022-03-15
  11  

In [37]:
os.chdir('..')
direct_df.to_csv('direct.csv', index=False)
display_df.to_csv('display.csv', index=False)
email_df.to_csv('email.csv', index=False)
organic_search_df.to_csv('organic_search.csv', index=False)
paid_search_df.to_csv('paid_search.csv', index=False)
referral_df.to_csv('referral.csv', index=False)
social_df.to_csv('social.csv', index=False)